In [ ]:
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from perceptronac.convex_hull import convex_hull

In [2]:
data = pd.read_csv("/home/lucas/Documents/perceptronac/results/exp_1655044978/exp_1655044978_train_values.csv",index_col="topology")
normalized_data = data.copy()
normalized_data["complexity"] = normalized_data["complexity"].apply(lambda x : math.log10(x)) # complexity in logarithmic scale
normalized_data = MinMaxScaler().fit_transform(normalized_data.values) # make the coordinates in the range [0,1]

In [1]:
plt.plot(normalized_data[:,0],normalized_data[:,1],linestyle="",marker="x") # verify that everything is alright

In [2]:
chull = normalized_data[convex_hull(normalized_data.tolist()),:]
plt.plot(normalized_data[:,0],normalized_data[:,1],linestyle="",marker="x")
plt.plot(chull[:,0],chull[:,1],linestyle="solid",color="red",marker=None)


In [3]:
data.iloc[convex_hull(normalized_data.tolist()),:]